In [3]:
import json
import requests
import re,string,timeit
import time
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np

Setup your parameters
Request URL
https://api.domain.com.au/v2/suburbPerformanceStatistics/NSW/Castle%20Hill/2154?propertyCategory=house&bedrooms=3&periodSize=quarters&startingPeriodRelativeToCurrent=40&totalPeriods=40


Provide your client credentials as per your Domain developer account.

Required: client_id and client_secret

Make a POST request to receive token.

In [4]:
load_dotenv()
client_id =os.getenv("domain_client_id")
client_secret = os.getenv("domain_client_oauth_secrets")
print(client_secret)


secret_d6bb4c7e7b1ffd35b93e894759a0eb1d


In [5]:
#to authenticate client credentials
# Make a POST request to the https://auth.domain.com.au/v2/connect/token endpoint using your client_id and client_secret, along with the list of scopes that you require. 
auth_url2 = 'https://auth.domain.com.au/v1/connect/token'

In [6]:

# https://developer.domain.com.au/docs/v2/apis/pkg_properties_locations/references/suburbperformance_get_bynamedsuburb
# GET /v2/suburbPerformanceStatistics/{state}/{suburb}/{postcode}

In [7]:
#Scope and Permissions
#The following scopes may be used when calling this endpoint
#https://developer.domain.com.au/docs/v1/apis/pkg_properties_locations/references/properties_get:

scopes = ['api_suburbperformance_read']

In [8]:
url_endpoint2 = 'https://api.domain.com.au/v2/suburbPerformanceStatistics/'

In [9]:
# TEST API : to define suburb details example for 1 suburb only ( this suburb has been tested on livebrowser)
# state='NSW'
# suburb="Chatswood"
# postcode='2067'
# # property_category = 'house'
# period_size = 'quarters'
# period_length = 40
# total_periods= 40
# url_detailed=f'https://api.domain.com.au/v2/suburbPerformanceStatistics/{state}/{suburb}/{postcode}/'


# url_detailed = 'https://api.domain.com.au/v2/suburbPerformanceStatistics/{}/{}/{}?propertyCategory={}&bedrooms={}&periodSize={}&startingPeriodRelativeToCurrent={}&totalPeriods={}'.format(state,suburb,postcode,propertyCategory,bedrooms,periodSize,startingPeriodRelativeToCurrent,totalPeriods)

In [12]:

df=pd.read_csv("nsw_data.csv",index_col="Suburb")


In [13]:
#select suburbs starts from 2000 Sydney to eliminate postcodes in our df which are PO BOX
mask1=df['Postcode']>1999
mask2=df['Postcode']<2080
list_suburbs=df[mask1&mask2]
# We, humans are biased, so let the Algorithm make the choice
list_suburbs=list_suburbs.loc[np.random.choice(list_suburbs.index,10,replace=False)]
list_suburbs=list_suburbs.to_dict()["Postcode"]
list_suburbs
# len(list_suburbs)

{'Double Bay': 2028,
 'Lane Cove': 2066,
 'Wahroonga': 2076,
 'Bondi Junction Plaza': 2022,
 'Banksmeadow': 2019,
 'North St Ives': 2075,
 'Rose Bay North': 2030,
 'Waterloo': 2017,
 'Matraville': 2036,
 'Bobbin Head': 2074}

# list_suburbs['suburb']
final_list_suburbs

In [15]:
final_list_suburbs = []

In [16]:
#housing data with bedrooms, houseCategories, timeperiods length
def get_property_info(suburb,postcode):
    response = requests.post(auth_url2,
        data = {
        'client_id':client_id,
        'client_secret':client_secret,
        'grant_type':'client_credentials',
        'scope': scopes,
        'Content-Type':'text/json'}
        )
    json_res = response.json()
    access_token=json_res["access_token"]
    print(access_token)
    auth = {'Authorization':'Bearer '+access_token}
    
    url_detailed=f'https://api.domain.com.au/v2/suburbPerformanceStatistics/NSW/{suburb}/{postcode}/?totalPeriods=40'

    res2 = requests.get(url_detailed,headers = auth)
    r2=res2.json()
    json_object =json.dumps(r2,indent=4)
    json.dumps(r2,indent=4)
            
    if "header"  in json.loads(json_object).keys(): 

        with open("./resources/{}_{}.json".format(postcode,suburb),"w") as outfile:
            outfile.write(json_object)   
    else:

        print('Failure in',suburb, postcode,e,access_token)






In [19]:
#  final_list_suburbs.append(suburb)
for suburb,postcode in list_suburbs.items():
    try:
        get_property_info(suburb,postcode)
        print('Success', suburb, postcode)
        final_list_suburbs.append(suburb)

    except Exception as e:
       
        print('Failure in',suburb, postcode,e)
        pass
    
   

Failure in Double Bay 2028 'access_token'
Failure in Lane Cove 2066 'access_token'
Failure in Wahroonga 2076 'access_token'
Failure in Bondi Junction Plaza 2022 'access_token'
Failure in Banksmeadow 2019 'access_token'
Failure in North St Ives 2075 'access_token'
Failure in Rose Bay North 2030 'access_token'
Failure in Waterloo 2017 'access_token'
Failure in Matraville 2036 'access_token'
Failure in Bobbin Head 2074 'access_token'


In [18]:
len(final_list_suburbs)

0

In [15]:
#to open json file saved in previous step & convert to dataframe & combine all dataframe into a master dataframe
df = pd.DataFrame()
frames= []
for suburb in final_list_suburbs: 
    with open("./resources/{}_{}.json".format(list_suburbs[suburb],suburb),"r") as sub_housing_data:
        sub_housing_data=json.load(sub_housing_data)
        suburb_name=sub_housing_data['header']['suburb']
        
    for x in range (40):
        
        headers_dict1=sub_housing_data['series']['seriesInfo'][x]['values']
        headers_dict1["year"]=sub_housing_data['series']['seriesInfo'][x]["year"]
        headers_dict1["month"]=sub_housing_data['series']['seriesInfo'][x]["month"]
        sub_df1 = pd.DataFrame(headers_dict1.values(),index=headers_dict1.keys(),columns=[suburb_name]).T
      
        sub_df1 = sub_df1.reset_index().drop(columns=["auctionNumberWithdrawn","daysOnMarket","discountPercentage",
                                                      "5thPercentileSoldPrice","25thPercentileSoldPrice",
                                                      "75thPercentileSoldPrice","95thPercentileSoldPrice",
                                                      "numberRentListing","highestRentListingPrice","lowestRentListingPrice"]).rename(columns={"index":"suburb"})
        frames.append(sub_df1)
df = pd.concat(frames)
        

    

In [16]:
# df.head(10)
# suburb_dfs={}
# for sub in set(df['suburb']):
#     sub_df=df[df['suburb']==sub]
#     suburb_dfs[sub]=sub_df

,suburb,medianSoldPrice,numberSold,highestSoldPrice,lowestSoldPrice,medianSaleListingPrice,numberSaleListing,highestSaleListingPrice,lowestSaleListingPrice,auctionNumberAuctioned,auctionNumberSold,medianRentListingPrice,year,month
0,Botany,750000.0,29.0,1370000.0,294000.0,750000.0,46.0,3500000.0,540000.0,18.0,11.0,630.0,2011.0,10.0
0,Botany,720000.0,26.0,1215000.0,585000.0,750000.0,43.0,1200000.0,539000.0,12.0,6.0,680.0,2012.0,1.0
0,Botany,716000.0,35.0,1130000.0,510000.0,755000.0,46.0,3500000.0,539000.0,16.0,10.0,650.0,2012.0,4.0
0,Botany,730000.0,35.0,1250000.0,512000.0,770000.0,43.0,2700000.0,520000.0,17.0,10.0,655.0,2012.0,7.0
0,Botany,787000.0,36.0,1700000.0,270000.0,775000.0,53.0,1800000.0,500000.0,12.0,7.0,675.0,2012.0,10.0
0,Botany,776000.0,18.0,1210000.0,570000.0,773000.0,41.0,1700000.0,501000.0,10.0,5.0,740.0,2013.0,1.0
0,Botany,867000.0,31.0,1120000.0,640000.0,740000.0,41.0,2000000.0,135000.0,18.0,13.0,750.0,2013.0,4.0
0,Botany,768000.0,32.0,1470000.0,599000.0,850000.0,38.0,1500000.0,356000.0,14.0,10.0,665.0,2013.0,7.0
0,Botany,900000.0,39.0,1410000.0,555000.0,917000.0,37.0,1500000.0,356000.0,14.0,11.0,690.0,2013.0,10.0
0,Botany,940000.0,27.0,1450000.0,650000.0,970000.0,30.0,1628000.0,356000.0,21.0,14.0,700.0,2014.0,1.0


In [25]:
df.columns

Index(['year', 'month', 'suburb', 'medianSoldPrice', 'numberSold',
       'highestSoldPrice', 'lowestSoldPrice', 'medianSaleListingPrice',
       'numberSaleListing', 'highestSaleListingPrice',
       'lowestSaleListingPrice', 'auctionNumberAuctioned', 'auctionNumberSold',
       'medianRentListingPrice'],
      dtype='object')

In [26]:
df = df[['year', 'month','suburb', 'medianSoldPrice', 'numberSold', 'highestSoldPrice',
       'lowestSoldPrice', 'medianSaleListingPrice', 'numberSaleListing',
       'highestSaleListingPrice', 'lowestSaleListingPrice',
       'auctionNumberAuctioned', 'auctionNumberSold', 'medianRentListingPrice',
       ]]

In [27]:
df.sample(10)

,year,month,suburb,medianSoldPrice,numberSold,highestSoldPrice,lowestSoldPrice,medianSaleListingPrice,numberSaleListing,highestSaleListingPrice,lowestSaleListingPrice,auctionNumberAuctioned,auctionNumberSold,medianRentListingPrice
0,2018.0,3.0,Towrang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2013.0,3.0,Baryulgil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021.0,6.0,Manly Vale,2805000.0,11.0,3825000.0,1525000.0,NaN,8.0,2750000.0,2200000.0,6.0,6.0,1100.0
0,2011.0,9.0,Clydesdale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2015.0,9.0,Cudgegong,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2013.0,12.0,Timbumburi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021.0,6.0,Hat Head,NaN,NaN,NaN,NaN,NaN,2.0,1395000.0,750000.0,NaN,NaN,NaN
0,2011.0,12.0,Koraleigh,NaN,NaN,NaN,NaN,NaN,1.0,120000.0,120000.0,NaN,NaN,NaN
0,2020.0,9.0,Clydesdale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2016.0,6.0,Stroud Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df.to_csv('data_V2_25082021.csv',index=False)